# Vent Validation

Validation of Vent produced data vs. AD Instruments


In [ ]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import sys
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import scipy.signal as signal
import scipy.interpolate as interpolate
from scipy.spatial import distance

def moving_avg_rms(data, window_size): # Calculate RMS
    # Moving avg RMS: https://stackoverflow.com/questions/8245687/numpy-root-mean-squared-rms-smoothing-of-a-signal
    data2 = np.power(data,2)
    window = np.ones(window_size)/float(window_size)
    return(np.sqrt(np.convolve(data2, window, 'valid')))

def butter(data, filter_order, cutoff, sampling_freq):
    b, a = signal.butter(filter_order, cutoff, btype='lowpass', analog=False, fs = sampling_freq)
    #return signal.lfilter(b, a, data) 
    return  signal.filtfilt(b, a, data) 

def smooth(y, box_pts):
    # unlike rms smoothing will preserve sign
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

def volume(flow, sample_rate, min_samples=50):
    vol = []
    tid = []
    v = 0    # running sum of volume
    t = 0    # running sum of tidal
    lt = 0   # calculated tidal at last breath
    p = 0    # state position
    s = 0    # state samples
    for f in flow:
        pos = (f >= 0)
        s += 1
        if pos != p and s < min_samples:
            # not enough samples since last switch
            pass
        elif pos != p:
            p = pos
            s = 0
            # reset on switch to positive direction
            if p == True:
                v = 0
                lt = t
                t = 0
        if p == False:
            t -= f
            
        v += f
        vol.append(v)
        tid.append(lt)
        
    return {
        'volume' : np.array(vol) / sample_rate,
        'tidal'  : np.array(tid) / sample_rate
    }

def save_text(txt, width, height):
    fig = plt.figure()
    fig.set_size_inches(width, height, forward = False)
    # fig.clf()
    fig.text(0.5,0.5,txt, transform=fig.transFigure, size=24, ha="center")
    save_fig(fig)
            
def save_fig(fig, title=""):
    if report:
        fig.suptitle(title, fontsize=16)
        report.savefig(fig)
    
    
def _plot(ndarray, components, offset, off, axs, legend, y_column, style='-'):
    ndkeys = ndarray.dtype.names
    comps = components if components else [ k for k in ndkeys if k != y_column]
    for c in comps:
        offset += off
        x = ndarray[y_column]
        y = ndarray[c] + offset
        axs.plot(x, y, style)
    axs.set_ylabel(f"{c}")
    return offset

    
def simple_plot(axs, streams, components=None, off=0, y_column='elapsed_s', title='', leg_contents=[], style='-'):

    if type(streams) != type([]):
        streams =[streams]
    
    offset = 0
    legend = []    
    for idx, stream in enumerate(streams):
        offset = _plot(stream, components, offset, off, axs, legend, y_column, style=style)
        if idx < len(leg_contents):
            legend.append(leg_contents[idx])
        
    axs.legend(legend, frameon=False, loc='lower right')
    axs.spines['top'].set_visible(False)
    axs.spines['right'].set_visible(False)
    axs.set_xlabel(f"{y_column}")
    axs.set_title(title)

def delta(compare, truth, field):
    # interpolate 
    orig = np.linspace(0,1,len(truth[field]))
    interp = np.linspace(0,1,len(compare[field]))
    truth_i = np.interp(interp, orig, truth[field])
    #print(f"orig:{len(truth[field])} interp:{len(truth_i)}")
    #print(signal.correlate(compare[field], truth_i))
    return (np.sum(compare[field] - truth_i) / np.sum(truth_i))
    


In [ ]:
collection = '../vent/peeper/ISO1'
offset = 4.9 # VCV1
cutoff = -0.1

"""
collection = '../vent/peeper/ISO2'
offset = -0.4 # VCV2
cutoff = -0.1

collection = '../vent/peeper/ISO3'
offset = -238.2 # VCV2
cutoff = -0.1

collection = '../vent/peeper/P15'
offset = -470.2 # VCV2
cutoff = -0.1

collection = '../vent/peeper/R30'
offset = 1.6 # VCV2
cutoff = -0.1

collection = '../vent/peeper/C200'
offset = -29.7 # VCV2
cutoff = -0.1

collection = '../vent/peeper/ISO_PCV1'
offset = 5.4 # PCV1
cutoff = -0.1


collection = '../vent/peeper/ISO_PCV2'
offset = 16.0 # PCV2
cutoff = -0.1
"""

ad_data = collection + '.txt'
ci_data = collection + '.out'
report = PdfPages(collection + '.pdf')

## AD data

Read in data exported from labchart

In [ ]:
# read lab chart exported data
ad_df = pd.read_csv(ad_data, delimiter='\t', names=["ts", "flow", "pressure", "volume", "rr", 
                                                    "max_flow", "min_flow", "max_pressure", "min_pressure",
                                                    "tidal", "minute"])

# calculate sample rate
ad_time = np.max(ad_df['ts'])-np.min(ad_df['ts'])
ad_sample_rate = ad_df.shape[0]/ad_time
print(f"AD sample rate {ad_sample_rate}")

# conversions
ad_df["flowmin"] = ad_df["flow"] * 60.0
ad_df["flowagg"] = ad_df["flowmin"]
ad_df["tidal"] = ad_df["tidal"] * 1000    # convert from L to ml
ad_df["volume"] = ad_df["volume"] * 1000  # convert from L to ml

# convert to numpy records for plotting
ad_np = ad_df.to_records(index=False)


## VOX Data

Read in data exported from VOX vent

In [ ]:
# read exported data 
ci_df = pd.read_csv(ci_data, delimiter=' ', names=["epoch","flowmin","volume", "tidal","in_p1","in_p2","in_flowmin","ex_p1","ex_p2","ex_flowmin"])
start_time = ci_df["epoch"].iloc[0]

# calculate sample rate
ci_time = np.max(ci_df['epoch'])-np.min(ci_df['epoch'])
ci_sample_rate = ci_df.shape[0]/ci_time
print(f"CI sample rate {ci_sample_rate}")

# inflow exflow and flow (composite) in LPS
ci_df["inflow"] = butter(ci_df["in_flowmin"], 7, 10, ci_sample_rate) / 60.0
ci_df["exflow"] = -butter(ci_df["ex_flowmin"], 7, 10, ci_sample_rate) / 60.0
ci_df["flow"] = butter(ci_df["flowmin"], 7, 10, ci_sample_rate) / 60

# vox displayed pressure is ex p2
ci_df["pressure"] = ci_df["ex_p2"]

# trim array to size of ad report
ci_df["ts"] = ci_df["epoch"] - start_time + offset
ci_df = ci_df.query(f'ts>{min(ad_df["ts"])} and ts < {max(ad_df["ts"])}')
ci_np = ci_df.to_records(index=False)



## AD Labchart Report

Graph reported stats from labchart report  

In [ ]:

fig, axs = plt.subplots(ncols=1, nrows=5, constrained_layout=True)
fig.set_size_inches(15, 9, forward = False)
simple_plot(axs[0], [ad_np], components=["flow"], y_column="ts", leg_contents=["Flow (LPS)"], style="r-")
simple_plot(axs[1], [ad_np], components=["pressure"], y_column="ts", leg_contents=["Pressure (cmH20)"], style="b-")
simple_plot(axs[2], [ad_np], components=["volume"], y_column="ts", leg_contents=["Volume (ml)"], style="g-")
simple_plot(axs[3], [ad_np], components=["tidal"], y_column="ts", leg_contents=["Tidal Volume (ml)"], style="g-")
simple_plot(axs[4], [ad_np], components=["minute"], y_column="ts", leg_contents=["Minute Ventilation (L)"], style="m-")
save_fig(fig, title="AD Labchart Report")

## Pressure Validation

Comparison of VOX pressure calculation and that of AD Instruments test rig

In [ ]:
fig, axs = plt.subplots(ncols=1, nrows=1, constrained_layout=True)
fig.set_size_inches(15, 10, forward = False)
pressure_delta = delta(ci_np, ad_np, "pressure")*100
simple_plot(axs, [ad_np, ci_np], components=["pressure"], y_column="ts", leg_contents=["AD Pressure (cmH20)", f"VOX Pressure ({pressure_delta:+.2f}%)"])
save_fig(fig, title="Pressure Validation")

## Comparison of Tidal Volumes

[Definitions](http://www.anaesthesia.med.usyd.edu.au/resources/lectures/ventilation_clt/ventilation.html) 
> Tidal Volume (Vt) is the amount of gas expired per breath - typically 500ml at rest.
>
> Deadspace Volume (VD) is the sum of the Anatomic Deadspace, due to the volume of the airways (typically 150ml), and Physiologic Deadspace, due to alveoli which are ventilated but not perfused (usually insignificant).
>
> Minute Volume (VE) is the amount of gas expired per minute.

    
Here we compare Tidal Volumes

1. AD Instruments calculate Tidal Volume
1. VOX calculation of Tidal Volume (calculated between air off and peep closed)
1. Post Processed AD - to make sure our math is not fundamentally different from AD




In [ ]:
# on device calculation of ex-flow 
ci_ex_flow_df = ci_df.copy()
ci_ex_flow_df.loc[ci_ex_flow_df.flow > cutoff, 'flow'] = 0
ci_vs = volume(ci_ex_flow_df["flow"], ci_sample_rate)
ci_ex_flow_df["tidal"] = ci_vs["tidal"] * 1000

# ad calculation of ex-flow 
ad_ex_flow_df = ad_df.copy()
ad_ex_flow_df.loc[ad_ex_flow_df.flow > cutoff, 'flow'] = 0
ad_vs = volume(ad_ex_flow_df["flow"], ad_sample_rate)
ad_ex_flow_df["tidal"] = ad_vs["tidal"] * 1000

flow_delta = delta(ci_ex_flow_df, ad_ex_flow_df, "flow")*100
ci_tidal = np.mean(ci_ex_flow_df["tidal"])
ad_tidal = np.mean(ad_ex_flow_df["tidal"])
print(f"AD Mean Tidal : {ad_tidal}  CI Mean Tidal : {ci_tidal}")
tidal_delta = delta(ci_ex_flow_df, ad_ex_flow_df, "tidal")*100

fig, axs = plt.subplots(ncols=1, nrows=2, constrained_layout=True)
fig.set_size_inches(15, 10, forward = False)

simple_plot(axs[0], [ad_ex_flow_df.to_records(), ci_ex_flow_df.to_records()], components=["flow"], y_column="ts", 
            leg_contents=["AD Expiry Flow (LPS)", f"VOX Expiry Flow ({flow_delta:+.2f}%)"])
simple_plot(axs[1], [ad_ex_flow_df.to_records(), ci_ex_flow_df.to_records(), ad_np], components=["tidal"], y_column="ts", 
            leg_contents=["AD Tidal Calculated (L)", f"VOX Tidal ({tidal_delta:+.2f}%)", "AD Tidal (L)"])
save_fig(fig, title="Expiry Flow and Tidal Volume Validation")


In [ ]:
report.close()
report = None

## 